In [1]:
import numpy as np


In [2]:
import os.path as osp
import cv2
import numpy as np

import sys
sys.path.append('../../../') 

from dust3r.datasets.base.base_stereo_view_dataset import BaseStereoViewDataset
from dust3r.utils.image import imread_cv2


In [8]:
from itertools import combinations
from collections import defaultdict
from tqdm.auto import tqdm
from scipy.spatial import cKDTree
from sklearn.neighbors import NearestNeighbors

def compute_iou(pointcloud1, pointcloud2, pixels_count=1000, distance_threshold=0.4):
    nbrs = NearestNeighbors(n_neighbors=1, algorithm = 'kd_tree').fit(pointcloud2)
    distances, indices = nbrs.kneighbors(pointcloud1)
    intersection1 = np.count_nonzero(distances.flatten() < distance_threshold)
    
    # nbrs = NearestNeighbors(n_neighbors=1, algorithm = 'kd_tree').fit(pointcloud1)
    # distances, indices = nbrs.kneighbors(pointcloud2)
    # intersection2 = np.count_nonzero(distances.flatten() < distance_threshold)
    
    # iou = min(intersection1 / pixels_count, intersection2 / pixels_count)
    iou = intersection1 / len(pointcloud1)
    if iou > 0.75:
        return 0.0

    return intersection1

def simulate_occlusion(point_cloud, radius):
    """
    Simulates occlusion by attaching a ball of fixed radius to each 3D point.
    Points occluded by another point within the radius are removed.
    """
    if len(point_cloud.shape) == 3:
        point_cloud = point_cloud.reshape(-1, 3)
    return point_cloud
    # tree = cKDTree(point_cloud)
    # 
    # indices = tree.query_ball_tree(tree, radius)
    # 
    # visible_mask = np.full(len(point_cloud), True, dtype=bool)
    # for i, neighbors in enumerate(indices):
    #     if visible_mask[i]:
    #         for neighbor in neighbors[1:]:
    #             visible_mask[neighbor] = False
    # 
    # visible_points = point_cloud[visible_mask]
    # return visible_points

def compute_iou_with_occlusion(pc1, pc2, radius=0.0005):
    """
    Computes the IoU between two 3D point clouds with artificial occlusion.
    """
    visible_pc1 = simulate_occlusion(pc1, radius)
    visible_pc2 = simulate_occlusion(pc2, radius)
    
    return compute_iou(visible_pc1, visible_pc2, pixels_count=pc1.shape[0] * pc1.shape[1])

def quality_pair_score(iou, alpha):
    """
    Compute the quality pair score s = IoU × 4 cos(α)(1 - cos(α))
    """
    angle_term = 4 * np.cos(alpha) * (1 - np.cos(alpha))
    return iou * angle_term if angle_term > 0 else 0

def select_best_pairs(dataset, iou_threshold=0.75, score_threshold=0.001, image_threshold=50):
    """
    Select the best image pairs using a greedy algorithm
    """
    # pairs = list(combinations(dataset, 2))
    pair_scores = []

    for i in range(20):
        img1 = dataset[i][0]
        img2 = dataset[i][1]
        
        iou = compute_iou_with_occlusion(img1['pts3d'], img2['pts3d'])
        if iou == 0:
            continue
        
        pose1, pose2 = img1['camera_pose'], img2['camera_pose']
        rotation_diff = pose1[:3, :3].T @ pose2[:3, :3]  # Relative rotation matrix
        alpha = np.arccos(np.clip((np.trace(rotation_diff) - 1) / 2, -1, 1))  # Angle in radians
    
        score = quality_pair_score(iou, alpha)
        if score > score_threshold:
            pair_scores.append((score, img1, img2))

    pair_scores.sort(key=lambda x: x[0], reverse=True)

    selected_pairs = []
    used_images = defaultdict(int)

    for score, img1, img2 in pair_scores:
        if used_images[img1['idx']] > image_threshold  or used_images[img2['idx']] > image_threshold:
            continue

        selected_pairs.append((img1, img2))
        used_images[img1['idx']] += 1
        used_images[img2['idx']] += 1
        
    return selected_pairs

selected_pairs = select_best_pairs(dataset)
print(len(selected_pairs))
for img1, img2 in selected_pairs[:10]:
    print(f"Selected pair: Image {img1['idx']} and Image {img2['idx']}")

10
Selected pair: Image (17, 0, 0) and Image (17, 0, 1)
Selected pair: Image (11, 0, 0) and Image (11, 0, 1)
Selected pair: Image (14, 0, 0) and Image (14, 0, 1)
Selected pair: Image (15, 0, 0) and Image (15, 0, 1)
Selected pair: Image (16, 0, 0) and Image (16, 0, 1)
Selected pair: Image (3, 0, 0) and Image (3, 0, 1)
Selected pair: Image (10, 0, 0) and Image (10, 0, 1)
Selected pair: Image (2, 0, 0) and Image (2, 0, 1)
Selected pair: Image (5, 0, 0) and Image (5, 0, 1)
Selected pair: Image (8, 0, 0) and Image (8, 0, 1)


In [ ]:
selected_pairs

In [60]:
for img1, img2 in selected_pairs:
    # print(img1[0])
    print(f"Selected pair: Image {img1[0]['idx']} and Image {img2[0]['idx']}")

In [5]:
from dust3r.datasets import UnderWaterDataset

In [6]:
dataset = UnderWaterDataset(split='train', ROOT='/home/aleksandra/dense_glomap_output', resolution=224)

In [9]:
import torch

seed = torch.initial_seed()  # this is different for each dataloader process
rng = np.random.default_rng(seed=seed)
dataset._create_pairs_indexes((224, 224), rng, pairs_per_image=30, image_threshold=50)

1082


[(18, 37),
 (24, 39),
 (18, 36),
 (23, 36),
 (24, 38),
 (21, 37),
 (24, 40),
 (23, 39),
 (19, 37),
 (21, 36),
 (19, 38),
 (19, 36),
 (23, 38),
 (18, 26),
 (20, 37),
 (18, 27),
 (23, 37),
 (24, 36),
 (22, 36),
 (21, 38),
 (18, 35),
 (18, 38),
 (20, 38),
 (22, 37),
 (24, 37),
 (20, 36),
 (23, 35),
 (23, 40),
 (18, 28),
 (17, 27),
 (24, 35),
 (22, 38),
 (25, 39),
 (22, 35),
 (25, 38),
 (51, 61),
 (38, 41),
 (17, 28),
 (21, 35),
 (17, 26),
 (25, 37),
 (25, 40),
 (25, 36),
 (19, 26),
 (52, 61),
 (18, 29),
 (37, 41),
 (51, 62),
 (38, 42),
 (52, 62),
 (17, 22),
 (18, 25),
 (18, 34),
 (50, 61),
 (19, 35),
 (21, 39),
 (18, 33),
 (19, 27),
 (24, 41),
 (37, 42),
 (25, 35),
 (19, 39),
 (20, 35),
 (17, 29),
 (22, 34),
 (24, 43),
 (23, 34),
 (17, 31),
 (20, 26),
 (21, 27),
 (24, 33),
 (24, 34),
 (23, 41),
 (17, 21),
 (20, 39),
 (18, 22),
 (22, 33),
 (22, 39),
 (39, 41),
 (17, 30),
 (20, 27),
 (53, 61),
 (34, 41),
 (17, 32),
 (22, 28),
 (26, 38),
 (19, 25),
 (21, 26),
 (18, 30),
 (24, 44),
 (22, 29),

In [9]:
type(dataset[0][0]['pts3d'])

numpy.ndarray

In [42]:
selected_pairs = select_best_pairs(dataset)
for img1, img2 in selected_pairs:
    print(f"Selected pair: Image {img1['idx']} and Image {img2['idx']}")

float32 float32


TypeError: unhashable type: 'numpy.ndarray'